# **🚌 Bus Depot Problem**

**🧠 Algorithm:** Vogel's Approximation Method with a Total Cost Matrix

### **⚙️ Process**

1. **Initialization**: 
   * Set up supply (buses), demand (depots), and the cost matrix. 🚌 
2. **Penalty Calculation**:  
   * For each row/column, find the difference between the two lowest costs. This shows potential savings! 📉
3. **Allocation**: 
   * Pick the row/column with the biggest penalty.
   * Assign as much as you can to the cheapest spot in that row/column.
   * Update supply and demand.  ✅
4. **Iteration**: Repeat steps 2 and 3 until everything is assigned. 🔁 
5. **Total Cost Calculation**: Add up the costs of all the assignments. 💰

### **📦 Imported Packages**

* **pandas**: To work with data tables (DataFrames) 📊
* **numpy**: For number crunching  and arrays 🔢
* **random**: To add some randomness  to the simulation🎲
* **rich** (Maybe): For making the output look fancier ✨

### **📝 Variable Description**

* **no_of_depots (int)**: How many depots we have  🏭
* **depot_capacities (list)**: How much space each depot has 📦
* **df_demand (dictionary)**: How many buses each depot needs (key: depot name, value: demand)   {'D1': 20, 'D2': 35, ...} 🚌 
* **total_demand (int)**: Total buses needed across all depots 🚌🚌🚌...
* **df_supply (dictionary)**: Available buses (usually starts with 1 bus per entry) 🚌
* **total_supply (int)**:  Total number of available buses 🚌🚌🚌...
* **df_dkm (dataframe)**: Distances or costs to move an empty bus between locations 🛣️ 
* **vehicle_ages (list)**: How old each bus is 🚌👴
* **age (dictionary)**:  Maps each bus to its age 
* **kpl (list)**: Kilometers per liter (fuel efficiency) for each bus, based on age ⛽️
* **co2_emissions (list)**: CO2 emissions for each bus, based on age 💨
* **fc_kpl (dictionary)**: Fuel cost per kilometer for each bus 💸
* **co2_c (dictionary)**: Cost related to CO2 emissions for each bus 💸
* **df_doc (list)**: Depot operating costs  💼
* **df_tdkom (dataframe)**: The big one! Total cost to park a bus at each depot 💰


In [62]:

import pandas as pd
import numpy as np
import random
from rich import print
from collections import defaultdict
# Set seed for reproducibility
random.seed(12)
np.random.seed(12)


In [63]:
from rich.table import Table
from rich.console import Console

# Initialize the Rich console
console = Console()

In [64]:

# Initialize depots and their capacities
# Number of Depots = 20 and Capacities = random integer between 100 and 150
no_of_depots = 20
depot_capacities = [random.randint(100, 150) for _ in range(no_of_depots)]
df_demand = {'D{}'.format(i+1): cap for i, cap in enumerate(depot_capacities)}
total_demand = sum(depot_capacities)

In [65]:
# Create a new table
table = Table(show_header=True, header_style="#a2d2ff")
table.add_column("Depot", style="dim", width=12)
table.add_column("Capacity")

# Populate the table with depot capacities
for depot, capacity in df_demand.items():
    table.add_row(depot, str(capacity))

# Print the table
console.print(table)

# Print total depot capacities
console.print(f"Total Depot Capacities: [bold]{total_demand}[/bold]", style="#b0efeb")

┏━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Depot        ┃ Capacity ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ D1           │ 130      │
│ D2           │ 117      │
│ D3           │ 142      │
│ D4           │ 133      │
│ D5           │ 142      │
│ D6           │ 122      │
│ D7           │ 109      │
│ D8           │ 124      │
│ D9           │ 100      │
│ D10          │ 123      │
│ D11          │ 130      │
│ D12          │ 117      │
│ D13          │ 141      │
│ D14          │ 129      │
│ D15          │ 144      │
│ D16          │ 138      │
│ D17          │ 114      │
│ D18          │ 135      │
│ D19          │ 100      │
│ D20          │ 142      │
└──────────────┴──────────┘

Total Depot Capacities: 2532

In [66]:

# Calculate total supply

# Generate supply data
df_supply = {i: 1 for i in range(total_demand)}

total_supply = sum(df_supply.values())

In [67]:
from rich.console import Console
from rich.table import Table

console = Console()

table = Table(show_header=True, header_style="bold cyan")
table.add_column("Bus", style="#a9def9")
table.add_column("Value", style="#ffcad4")

for key, value in list(df_supply.items())[:5]:
    table.add_row(str(key), str(value))

table.add_row("...", "...")
table.add_row(str(total_supply - 1), str(df_supply[total_supply - 1]))
console.print(table)


┏━━━━━━┳━━━━━━━┓
┃ Bus  ┃ Value ┃
┡━━━━━━╇━━━━━━━┩
│ 0    │ 1     │
│ 1    │ 1     │
│ 2    │ 1     │
│ 3    │ 1     │
│ 4    │ 1     │
│ ...  │ ...   │
│ 2531 │ 1     │
└──────┴───────┘

In [68]:

# Generate random dead kilometre matrix
df_dkm = pd.DataFrame(np.random.uniform(0.5, 40, size=(total_supply, no_of_depots))).astype(float)


In [69]:

table = Table(show_header=True, header_style="bold #cdb4db")

for i in range(5):
    table.add_column(f"Depot {i+1}", style="#a9def9")

table.add_column("...", style="#a9def9")
table.add_column(f"Depot {no_of_depots}", style="#a9def9")


for index, row in df_dkm.head(5).iterrows():
    row_values = [f"{value:.2f}" for value in row[:5]]
    table.add_row(*row_values, "...", str(round(df_dkm[no_of_depots-1][index], 2)))
    
table.add_row("...", "...","...", "...","...", "...", "...")
for index, row in df_dkm.tail(2).iterrows():
    row_values = [f"{value:.2f}" for value in row[:5]]
    table.add_row(*row_values, "...", str(round(df_dkm[no_of_depots-1][index], 2)))
console.print(table)

┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━━┓
┃ Depot 1 ┃ Depot 2 ┃ Depot 3 ┃ Depot 4 ┃ Depot 5 ┃ ... ┃ Depot 20 ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━━┩
│ 6.59    │ 29.73   │ 10.90   │ 21.58   │ 1.08    │ ... │ 6.85     │
│ 30.70   │ 1.32    │ 5.84    │ 5.09    │ 12.74   │ ... │ 18.33    │
│ 16.32   │ 39.81   │ 7.51    │ 38.52   │ 17.06   │ ... │ 2.4      │
│ 23.11   │ 32.24   │ 4.99    │ 11.45   │ 25.85   │ ... │ 26.78    │
│ 25.68   │ 34.57   │ 37.69   │ 18.10   │ 26.96   │ ... │ 25.9     │
│ ...     │ ...     │ ...     │ ...     │ ...     │ ... │ ...      │
│ 18.22   │ 9.82    │ 15.40   │ 16.80   │ 3.59    │ ... │ 29.39    │
│ 30.02   │ 26.11   │ 15.21   │ 35.12   │ 24.73   │ ... │ 21.31    │
└─────────┴─────────┴─────────┴─────────┴─────────┴─────┴──────────┘

In [70]:

random.seed(12)
np.random.seed(12)
# Generate vehicle age
vehicle_ages = [random.randint(1, 3) for _ in range(total_supply)]
age = {i: age for i, age in enumerate(vehicle_ages)}

# Generate kpl (kilometers per liter) and co2 emissions based on age
kpl, co2_emissions = [], []
for v_age in vehicle_ages:
    random.seed(12)
    np.random.seed(12)
    if v_age == 1:
        kpl.append(random.uniform(5.1, 6))
        co2_emissions.append(round(random.uniform(515, 524), 2))
    elif v_age == 2:
        kpl.append(random.uniform(4.1, 5))
        co2_emissions.append(round(random.uniform(525, 534), 2))
    else:
        kpl.append(random.uniform(3.1, 4))
        co2_emissions.append(round(random.uniform(535, 540), 2))

# Calculate fuel consumption, co2 cost and Depot Operating Cost
fc_kpl = {i: (111 / kpl_val) for i, kpl_val in enumerate(kpl)}
co2_c = {i: (0.00118642 * co2_val) for i, co2_val in enumerate(co2_emissions)}
random.seed(12)
np.random.seed(12)
df_doc = [random.uniform(50, 100) for _ in range(no_of_depots)]

In [71]:
# Create a new table with pastel colors

table = Table(show_header=True, header_style="bold #ffb5a7")
table.add_column("Vehicle ID", style="dim", width=12)
table.add_column("KPL", style="#b0efeb")
table.add_column("CO2 Emissions", style="#a9def9")
table.add_column("Fuel Consumption (FC/KPL)", style="#cdb4db")
table.add_column("CO2 Cost", style="#ffcad4")

# Populate the table with vehicle data
for i in range(5):
    table.add_row(
        str(i),
        "{:.2f}".format(kpl[i]),
        "{:.2f}".format(co2_emissions[i]),
        "{:.2f}".format(fc_kpl[i]),
        "{:.5f}".format(co2_c[i])
    )

table.add_row("...", "...","...", "...","...")
table.add_row("...", "...","...", "...","...")

for i in range(total_supply-5, total_supply):
    table.add_row(
        str(i),
        "{:.2f}".format(kpl[i]),
        "{:.2f}".format(co2_emissions[i]),
        "{:.2f}".format(fc_kpl[i]),
        "{:.5f}".format(co2_c[i])
    )
    
console.print(table)

table = Table(show_header=True, header_style="bold cyan")
table.add_column("Depot", style="#a9def9")
table.add_column("Depot Operating Cost", style="#ffcad4")

for key, value in enumerate(df_doc[:3]):
    table.add_row(str(key), str(value))

table.add_row("...", "...")

for index, value in enumerate(df_doc[-3:], start=no_of_depots - 3):
    table.add_row(str(index), str(value))
    
console.print(table)

┏━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Vehicle ID   ┃ KPL  ┃ CO2 Emissions ┃ Fuel Consumption (FC/KPL) ┃ CO2 Cost ┃
┡━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 0            │ 4.53 │ 530.92        │ 24.52                     │ 0.62989  │
│ 1            │ 4.53 │ 530.92        │ 24.52                     │ 0.62989  │
│ 2            │ 3.53 │ 538.29        │ 31.47                     │ 0.63864  │
│ 3            │ 3.53 │ 538.29        │ 31.47                     │ 0.63864  │
│ 4            │ 3.53 │ 538.29        │ 31.47                     │ 0.63864  │
│ ...          │ ...  │ ...           │ ...                       │ ...      │
│ ...          │ ...  │ ...           │ ...                       │ ...      │
│ 2527         │ 3.53 │ 538.29        │ 31.47                     │ 0.63864  │
│ 2528         │ 5.53 │ 520.92        │ 20.08                     │ 0.61803  │
│ 2529         │ 3.53 │ 538.29        │ 31.47                     │ 0.63864  │
│ 2530         │ 5.53 │ 520.92        │ 20.08                     │ 0.61803  │
│ 2531         │ 4.53 │ 530.92        │ 24.52                     │ 0.62989  │
└──────────────┴──────┴───────────────┴───────────────────────────┴──────────┘

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Depot ┃ Depot Operating Cost ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ 0     │ 73.7285339344274     │
│ 1     │ 82.87362513286277    │
│ 2     │ 83.32052355624191    │
│ ...   │ ...                  │
│ 17    │ 90.94100639210401    │
│ 18    │ 53.730481681793876   │
│ 19    │ 84.34727520467709    │
└───────┴──────────────────────┘

In [72]:
# Convert fc_kpl and co2_c dictionaries to pandas Series for efficient operations
fc_kpl_series = pd.Series(fc_kpl)
co2_c_series = pd.Series(co2_c)

# Calculate the distance-based costs (fuel consumption and CO2 emissions) for each depot-destination pair
distance_based_costs = df_dkm.multiply(fc_kpl_series, axis='index') + df_dkm.multiply(co2_c_series, axis='index')

# Convert doc list to a pandas Series and align it with the columns of df_dkm for broadcasting
doc_series = pd.Series(df_doc, index=df_dkm.columns)

# Add the fixed depot operation costs to the distance-based costs
# Broadcasting ensures that doc_series values are added to each row corresponding to the depot
df_tdkom = distance_based_costs.add(doc_series, axis='columns')


In [73]:

table = Table(show_header=True, header_style="bold #cdb4db", title="[bold]🚌 Bus Depot Transportation Problem 🚌[/bold]")
table.add_column("", style="#a9def9", justify="center")
for i in range(5):
    table.add_column(f"Depot {i+1}", style="#a9def9", justify="center")
    
table.add_column("...", style="#a9def9")
table.add_column(f"Depot {no_of_depots}", style="#a9def9", justify="center")
table.add_column("Supply ▼", style="#a9def9", justify="center")


for index, row in df_tdkom.head(10).iterrows():
    row_values = [f"{value:.2f}" for value in row[:5]]
    table.add_row(f"[#ffcad4]Bus {index+1}[/#ffcad4]",*row_values, "...", str(round(df_tdkom[no_of_depots-1][index], 2)),"1")
    
table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')
table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')
table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')

for index, row in df_tdkom.tail(10).iterrows():
    row_values = [f"{value:.2f}" for value in row[:5]]
    table.add_row(f"[#ffcad4]Bus {index+1}[/#ffcad4]",*row_values, "...", str(round(df_tdkom[no_of_depots-1][index], 2)),"1")
    
table.add_row("","───────", "───────","───────", "───────","───────", "", "───────","────────")
table.add_row("[#ffcad4]Demand ►[/#ffcad4]",*[str(value) for value in list(df_demand.values())[:5]],'...', str(list(df_demand.values())[-1]),str(total_supply))
console.print(table)

                          🚌 Bus Depot Transportation Problem 🚌                          
┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          ┃ Depot 1 ┃ Depot 2 ┃ Depot 3 ┃ Depot 4 ┃ Depot 5 ┃ ... ┃ Depot 20 ┃ Supply ▼ ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│  Bus 1   │ 239.45  │ 830.60  │ 357.47  │ 599.91  │  77.60  │ ... │  256.57  │    1     │
│  Bus 2   │ 845.80  │ 116.12  │ 230.21  │ 185.21  │ 370.96  │ ... │  545.24  │    1     │
│  Bus 3   │ 597.91  │ 1361.07 │ 324.58  │ 1294.06 │ 598.34  │ ... │  161.56  │    1     │
│  Bus 4   │ 815.77  │ 1118.04 │ 243.46  │ 424.66  │ 880.44  │ ... │  944.15  │    1     │
│  Bus 5   │ 898.19  │ 1192.80 │ 1293.66 │ 638.38  │ 916.31  │ ... │  915.83  │    1     │
│  Bus 6   │ 468.97  │ 687.25  │ 702.66  │ 665.35  │ 373.33  │ ... │  231.46  │    1     │
│  Bus 7   │ 240.84  │ 422.41  │ 737.06  │ 275.02  │ 150.42  │ ... │  352.81  │    1     │
│  Bus 8   │ 1048.51 │ 963.25  │ 1002.64 │ 238.48  │ 377.83  │ ... │  438.09  │    1     │
│  Bus 9   │ 585.85  │ 598.87  │ 104.59  │ 537.18  │ 696.79  │ ... │  870.99  │    1     │
│  Bus 10  │ 1002.03 │ 972.17  │ 146.60  │ 839.51  │ 168.31  │ ... │  763.4   │    1     │
│   ...    │   ...   │   ...   │   ...   │   ...   │   ...   │ ... │   ...    │    .     │
│   ...    │   ...   │   ...   │   ...   │   ...   │   ...   │ ... │   ...    │    .     │
│   ...    │   ...   │   ...   │   ...   │   ...   │   ...   │ ... │   ...    │    .     │
│ Bus 2523 │ 668.29  │ 616.76  │ 1056.44 │ 926.24  │ 791.57  │ ... │  866.32  │    1     │
│ Bus 2524 │ 366.92  │ 1342.35 │ 690.58  │ 1120.27 │ 348.67  │ ... │  523.74  │    1     │
│ Bus 2525 │ 456.10  │ 891.83  │ 659.16  │ 327.92  │ 740.42  │ ... │  710.25  │    1     │
│ Bus 2526 │ 515.35  │ 326.95  │ 203.94  │ 395.44  │ 635.12  │ ... │  345.72  │    1     │
│ Bus 2527 │ 475.57  │ 274.77  │ 824.86  │ 468.39  │ 358.38  │ ... │  116.88  │    1     │
│ Bus 2528 │ 612.29  │ 159.48  │ 283.93  │ 408.36  │ 110.33  │ ... │  834.51  │    1     │
│ Bus 2529 │ 125.82  │ 384.18  │ 660.79  │ 156.84  │ 434.82  │ ... │  401.5   │    1     │
│ Bus 2530 │ 551.66  │ 416.50  [0

In [74]:
# Transpose the total cost matrix to facilitate operations on columns (depots)
dk = df_tdkom.transpose()

# Convert the transposed dataframe into a dictionary for easier manipulation
# costs_t was used earlier for this variable
costs_t = dk.to_dict()

# Sort the depot names to maintain a consistent order for processing
# colsn was used earlier for this variable
colsn = sorted(df_demand.keys())
# Initialize a result dictionary to store the allocation results with default 0 values
# res was used earlier for this variable
res = dict((k, defaultdict(int)) for k in costs_t)
# Initialize a dictionary to store the sorted order of depots for each bus and vice versa based on cost
g = {}

# Populate the sorted order of depots for each bus based on ascending cost
for x in df_supply:
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])

# Populate the sorted order of buses for each depot based on ascending cost
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])
# Initialize a dictionary to store the sorted order of depots for each bus and vice versa based on cost
g = {}

# Populate the sorted order of depots for each bus based on ascending cost
for x in df_supply:
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])

# Populate the sorted order of buses for each depot based on ascending cost
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])
# Main loop for allocation using VAM until all supplies or demands are met
while g:
    d = {}  # Penalty for each depot
    for x in df_demand:
        d[x] = (costs_t[g[x][1]][x] - costs_t[g[x][0]][x]) if len(g[x]) > 1 else (costs_t[g[x][0]][x])
    
    s = {}  # Penalty for each bus
    for x in df_supply:
        s[x] = (costs_t[x][g[x][1]] - costs_t[x][g[x][0]]) if len(g[x]) > 1 else costs_t[x][g[x][0]]
    
    # Determine the bus or depot with the highest penalty to allocate next
    f = max(d, key=lambda n: d[n])  # Depot with the highest penalty
    t = max(s, key=lambda n: s[n])  # Bus with the highest penalty
    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)
    
    # Allocate as much as possible to the chosen bus-depot pair and adjust supply and demand
    v = min(df_supply[f], df_demand[t])
    res[f][t] += v
    df_demand[t] -= v

    # Remove the depot from consideration if its demand is fully met
    if df_demand[t] == 0:
        for k, n in df_supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del df_demand[t]

    # Remove the bus from consideration if its supply is fully allocated
    df_supply[f] -= v
    if df_supply[f] == 0:
        for k, n in df_demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del df_supply[f]
# Calculate the total cost of the final allocation
cost = 0
for g in sorted(costs_t):
    for n in colsn:
        y = res[g][n]
        cost += y * costs_t[g][n]

# Print the total cost of the transportation plan
print("Total Transportation Cost: ", cost)


KeyError: 'D1'